Loading Data

In [1]:
import pandas as pd

articles = pd.read_csv('articles.csv')

Preprocessing Data

In [2]:
import nltk
from datetime import datetime as dt
import pytz
from nltk.corpus import stopwords
nltk.download('stopwords')

# Credit to / help from https://saturncloud.io/blog/how-to-remove-stop-words-from-a-pandas-dataframe-using-python/
def remove_stopwords(words_tokenized):
    stop_words = set(stopwords.words('english'))  # List of english stopwords
    return [word for word in words_tokenized if word not in stop_words] # Using list comprehension, only choose the words that aren't stopwords

def convert_to_datetime(date_string):
    return dt.strptime(date_string, '%Y-%m-%d %H:%M:%S%z')

def preprocess(df, sample_size=None):
    df = df.dropna()
    # sample if specified
    if sample_size:
        df = df.sample(sample_size)
    # remove uncessary index column
    df = df.drop(df.columns[0], axis=1)
    # change stock column name to ticker
    df.rename(columns={'stock': 'ticker'}, inplace=True)
    # convert headlines to lowercase
    df['title'] = df['title'].str.lower()
    # remove punctuation
    df['title'] = df['title'].str.replace(r'[^a-zA-Z\s$0-9]', '', regex=True)
    # tokenize
    df['title'] = df['title'].str.split() 
    # remove stopwords
    df['title'] = df['title'].apply(remove_stopwords)
    # convert to datetime object
    df['date'] = df['date'].apply(convert_to_datetime)
    return df

articles = preprocess(articles, sample_size=1000)

articles

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cavingada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,date,ticker
1194513,"[stocks, set, new, 52week, high, yesterday, au...",2018-08-09 11:03:00-04:00,SYNL
591771,"[shares, several, basic, materials, companies,...",2020-03-31 11:50:00-04:00,HBM
427960,"[scotiabank, earlier, upgraded, enerplus, corp...",2016-12-13 10:14:00-05:00,ERF
145219,"[54, stocks, moving, wednesdays, midday, session]",2019-02-20 13:13:00-05:00,BDC
377947,"[morgan, stanley, downgrades, drilquip, inc, u...",2013-12-02 06:46:00-05:00,DRQ
...,...,...,...
715448,"[kimco, realty, corporation, reports, q4, ffo,...",2015-02-05 16:10:00-05:00,KIM
242255,"[afternoon, losers, march, 10th]",2014-03-10 15:18:00-04:00,CIMT
473759,"[top, 20, wall, street, analysts, 2015]",2015-12-17 10:44:00-05:00,FCSC
576350,"[stocks, hit, 52week, highs, friday]",2019-12-20 11:59:00-05:00,GSL


Grab Stock Returns

Based on the time of the article published, we will retrieve two adjusted close prices of the stock and compute the corresponding return.

If the time of the article is published before 4:00 P.M. (non-inclusive), then:
1. The 'before' price will be the most recent (before the date) trading day's adjusted close price
2. The 'after' price will be the most upcoming trading day's adjusted close price

If the time of the article is published after 4:00 P.M., then:
1. The 'before' price will be the same day's adjusted close price
2. The 'after' priec will be the next day's adjusted close price

In [3]:
#Get all the yfinance data we need based on date.
def retrieve_yfinance_data(row):
    curr_date = row['date']
    
    eod = dt.strptime('16:00:00', '%H:%M:%S').time()
    
    if curr_date.time() > eod:
        start_date = curr_date
        end_date = getValidTradingCloseDate(start_date, forward=True)
    else:
        end_date = curr_date
        start_date = getValidTradingCloseDate(end_date, forward=False)
        
    
    data = yf.download(row['ticker'], start=start_date, end=end_date + pd.Timedelta(days=1), interval='1m')
    if len(data) > 0:
        returns = (data['Adj Close'][-1] - data['Adj Close'][0]) / data['Adj Close'][0]
        
    return data

In [ ]:
import pandas_market_calendars as mcal
# The paramater forward is a boolean representing whether we are looking for the next valid trading day or the most recent trading day
def getValidTradingCloseDate(date, forward=True):
        nyse = mcal.get_calendar('NYSE')
        if forward:
            start_date = date
            end_date = date+date.dt.timedelta(days=15)
        else:
            start_date = date-dt.timedelta(days=15)
            end_date = date

        validTradingDays = nyse.valid_days(start_date=start_date , end_date=end_date)

        return validTradingDays.date[0] if forward else validTradingDays.date[-1]